In [1]:
import os
import MeCab
import pickle
import joblib
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [2]:
# ---------------------------------------------------------
# データの作成
# ---------------------------------------------------------
def create_data(author_name, txt_files):
  txt_dir = 'txt_' + author_name
  book_list = []
  content_list = []

  # ファイルの読み込みとコンテンツの保存
  for file_name in txt_files:
    file_path = os.path.join(txt_dir, file_name)
    with open(file_path, 'r', encoding='shift-jis') as file:
      book_list.append(file.readline().strip())                   # 書籍名を取得
      content_list.append(tokenize(sanitize(file.read())))        # 不要な部分を削除したコンテンツのトークンを取得

  return book_list, content_list

# ---------------------------------------------------------
# コンテンツの不要な部分を除外
# ---------------------------------------------------------
def sanitize(text):
  operations = [
    lambda text: re.split(r'\-{5,}', text)[2],
    lambda text: re.split(r'底本：', text)[0],
    lambda text: re.sub(r'《.+?》', '', text),
    lambda text: re.sub(r'［＃.+?］', '', text),
    lambda text: text.strip()
  ]

  for operation in operations:
    try:
      text = operation(text)
    except Exception as e:
      pass

  return text

# ---------------------------------------------------------
# 形態素解析
# ---------------------------------------------------------
def tokenize(text):
  tagger = MeCab.Tagger("")
  node = tagger.parseToNode(text)
  tokens = []

  while node is not None:
    part = node.feature.split(",")[0]
    if part in ["名詞", "動詞", "形容詞"]:
      tokens.append(node.surface)
    node = node.next

  return tokens

# ---------------------------------------------------------
# メイン
# ---------------------------------------------------------
with open("book_data.pkl", "rb") as f:
    data = pickle.load(f)

author_list = data["author_list"]
file_list   = data["file_list"]

tagged_data = []

# データの作成
for i in range(len(author_list)):
  book_list, content_list = create_data(author_list[i], file_list[i])
  for j in range(len(book_list)):
    tag = f'{author_list[i]}：{book_list[j]}'
    tagged_data.append({'tag': tag, 'content': ' '.join(content_list[j])})

# TF-IDF モデルの作成
corpus = [d['content'] for d in tagged_data]
vectorizer = TfidfVectorizer()
tfidf_matrix = vectorizer.fit_transform(corpus)

# インスタンスの保存
joblib.dump(vectorizer, "vectorizer.joblib")
joblib.dump(tfidf_matrix, "tfidf_matrix.joblib")

['tfidf_matrix.joblib']

In [3]:
# 新しいテキストのベクトル化と類似度の計算
new_text = '青びかる天弧のはてに、きらゝかに町はうかびて、六月のたつきのみちは、いまやはた尽きはてにけり。いさゝかの書籍とセロを、思ふまゝ'
new_text_tokens = tokenize(new_text)
new_text_vector = vectorizer.transform([' '.join(new_text_tokens)])

# コサイン類似度の計算
cosine_similarities = cosine_similarity(new_text_vector, tfidf_matrix).flatten()
similar_indices = cosine_similarities.argsort()[:-11:-1]

# 結果の表示
for idx in similar_indices:
  print(f"{tagged_data[idx]['tag']}：{cosine_similarities[idx]}")

宮沢 賢治：〔青びかる天弧のはてに〕：0.5338304133207578
宮沢 賢治：ありときのこ：0.026369219333433738
宮沢 賢治：青柳教諭を送る：0.025855718301230894
北原 白秋：思ひ出：0.02568242095534178
芥川 竜之介：芥川龍之介歌集：0.022983236274996113
宮沢 賢治：或る農学生の日誌：0.022447943278962598
宮沢 賢治：秋田街道：0.02041374320760784
ポー エドガー・アラン：ウィリアム・ウィルスン：0.0198302554455171
ポー エドガー・アラン：アッシャー家の崩壊：0.019343266417246836
芥川 竜之介：愛読書の印象：0.017117252667950132
